In [ ]:
%matplotlib widget
import warnings
import inspect
import matplotlib.pyplot as plt
import IPython.display
import numpy as np
from cued_sf2_lab.familiarisation import load_mat_img, plot_image
from cued_sf2_lab.laplacian_pyramid import quantise
from compression_schemes.dwt_funcs import *


# 10. Selection of preferred energy compaction options

The remainder of this project will concentrate on developing the rest of
an image compression system, based on a few of the filtering /
transformation schemes studied so far.

Since the subsequent processes are non-linear, we cannot expect to be able to
choose precisely the right front-end at this stage, so we adopt the pragmatic
approach of picking about three good candidates and trust that one of these
will lead to a near-optimum solution in the end. Remember that up to this point we have only been using entropy to give us an _estimate_ of the number of bits required, the accuracy of which is affected by subsequent stages.

At this stage it is worth trying your schemes with all three
test images, (`Lighthouse`, `Bridge`, and `Flamingo`). You will find `Bridge` more difficult to compress than the other two. You may also want to introduce other images of your own.

In [ ]:
lighthouse, _ = load_mat_img(img='lighthouse.mat', img_info='X')
bridge, _ = load_mat_img(img='bridge.mat', img_info='X')
flamingo, _ = load_mat_img(img='flamingo.mat', img_info='X')

In [ ]:
fig, axs = plt.subplots(1, 3)
plot_image(lighthouse, ax=axs[0])
plot_image(bridge, ax=axs[1])
plot_image(flamingo, ax=axs[2])

Write `.py` files to implement each of your
chosen schemes, so that you do not have to remember long sequences
of commands each time you run them. You can easily edit the M-files to introduce different options
later.  Using plenty of comments in these files will help when you want to change them.

In [ ]:
Xl = zero_mean(lighthouse)
Xb = zero_mean(bridge)
Xf = zero_mean(flamingo)

In [ ]:
Yl = nlevdwt(Xl, 5)
Yb = nlevdwt(Xb, 5)
Yf = nlevdwt(Xf, 5)

In [ ]:
scaled_l, Ylq, dwtent_l, Z_l = diff_step_sizes(Xl, 256, 5)
scaled_b, Ybq, dwtent_b, Z_b = diff_step_sizes(Xb, 256, 5)
scaled_f, Yfq, dwtent_f, Z_f = diff_step_sizes(Xf, 256, 5)

In [ ]:
fig, axs = plt.subplots(1, 3)
plot_image(Z_l, ax=axs[0])
plot_image(Z_b, ax=axs[1])
plot_image(Z_f, ax=axs[2])

In [ ]:
ratio_l = compression_ratio(Xl, Ylq, dwtent_l)
ratio_b = compression_ratio(Xb, Ybq, dwtent_b)
ratio_f = compression_ratio(Xf, Yfq, dwtent_f)
print(ratio_l, ratio_b, ratio_f)

# 11. Centre-clipped linear quantisers

The quantisers that you have used so far have all been uniform quantisers
(i.e.  all steps have been the same size).  However the probability
distributions of the intensities of the bandpass sub-images from the energy
compaction front-ends are usually highly peaked at zero.  The amount of data
compression depends heavily on the proportion of data samples which are
quantised to zero; if this approaches unity then high compression is
achieved.

Hence it is often found desirable to make the quantiser non-linear
so that more samples tend to be quantised to zero.  A simple way
to achieve this is to widen the step-size of the "zero" step.  In
a uniform quantiser, the "zero" step is normally centred on zero,
with rises to the next level at $\pm$ half of the step-size on
each side of zero. `quantise` allows a third argument `rise1` to be specified, which is
the point at which the first rise occurs on each side of the zero step.  A
value of `rise1` = `step/2` is the default, but `rise1` = $\left\{0.5, 1, 1.5\right\}\times$ `step` are worth investigating. To show what effect these have, try:

```python
x = np.arange(-100, 100+1)
y = quantise(x, 20, rise1)
fig, ax = plt.subplots()
ax.plot(x, y)
ax.grid()
```

In [ ]:
# your code here

import numpy as np

import matplotlib.pyplot as plt

 

x = np.arange(-100, 101)

step = 20

 

rise1_values = [0.5, 1.0, 1.5]

colors = ['r', 'g', 'b']

labels = [f"rise1 = {r} × step" for r in rise1_values]

 

fig, ax = plt.subplots()

 

for rise1, color, label in zip(rise1_values, colors, labels):

    y = quantise(x, step, rise1 * step)

    ax.plot(x, y, label=label, color=color)

 

ax.set_title("Effect of Varying rise1 on Quantiser Output")

ax.set_xlabel("Input x")

ax.set_ylabel("Quantised y")

ax.grid(True)

ax.legend()

plt.show()

A wider zero step means that more samples will be coded as zero and so the
entropy of the data will be reduced.  The use of a wide zero step is
beneficial if it results in a better entropy vs. error tradeoff than a uniform
quantiser.

<div class="alert alert-block alert-danger">

For each of your preferred front-end
schemes, investigate the effects of varying the first rise of the
quantiser.  To do this, you could plot how the quantising error
varies as a function of the number of bits for a few different ratios of
`rise1` to step-size, and hence find the ratio which gives the
best compression for a given rms error. </div>

In [ ]:
# You may wish to work in standalone python files (instead of notebooks) at this point!

Most current image compression standards use quantisers with a
double-width centre step (`rise1 = step`). Do not spend too much time
on this as the compression gains are likely to be quite small.

<div class="alert alert-block alert-danger">


Discuss whether your results indicate that `rise1 = step`
is a reasonable compromise if all quantisers are to be similar.
</div>

A final strategy which you can consider is to completely suppress some
sub-images or DCT coefficients.  This is equivalent to increasing `rise1`
to a very large value for these components.  In the sub-images / coefficients
which represent only the highest horizontal and vertical frequency components
combined, the effects of suppression can be almost unnoticable and yet
a useful saving in number of bits can be achieved.

<div class="alert alert-block alert-danger">

Investigate any additional gains which can be achieved with suppression
of some sub-images / coefficients.
</div>